# 📊 Repositório de Consultas SQL para Analistas de Dados: **Departamento ESG - Environmental (Meio Ambiente)**

ESG é um **conjunto de padrões e boas práticas que visa definir se uma empresa é socialmente consciente, sustentável e corretamente gerenciada**. Trata-se de uma forma de medir o desempenho de sustentabilidade de uma organização. A sigla, em inglês, reúne os três pilares desse movimento:
-	Environmental (meio ambiente)
-	Social
-	Governance (governança)

O critério ambiental inclui exigências nesse campo, como:
- a gestão de resíduos;
- a política de desmatamento (caso aplicável);
- o uso de fontes de energia renováveis pela empresa;

Além disso, o critério ambiental pode também se estender ao controle exercido pela empresa em terras que possui, se há ações para melhorar e preservar a biodiversidade, por exemplo.

Comentários sobre o Projeto:
- 🔍 Contribuições: Estamos abertos a contribuições da comunidade de analistas de dados! Se você tiver consultas SQL úteis que gostaria de compartilhar ou sugestões para melhorias, sinta-se à vontade para enviar um pull request. Juntos, podemos expandir e aprimorar este repositório para beneficiar toda a comunidade de análise de dados.
- 🎯 Sinta-se à vontade para explorar, utilizar e contribuir para este repositório, e que ele possa ser uma ferramenta valiosa em sua jornada como analista de dados!

# 📌 **KPI Departamento**
- Explicação KPI

```sql
SELECT
    column_1
FROM
    table_example
WHERE
    column_date BETWEEN '2024-01-01' AND '2024-12-31'
```